In [1]:
!pip install qiskit

In [ ]:
# unzipped already, no need to do again
!unzip pima-indians-diabetes-database.zip

pima-indians-diabetes-database.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  pima-indians-diabetes-database.zip
replace diabetes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# just in case someone wants to see the ugly comma seperated file
import csv
with open('/data/diabetes.csv', newline='') as File:  
    reader = csv.reader(File)
    for row in reader:
        print(row)

In [2]:
import numpy as np
data = np.genfromtxt('diabetes.csv', delimiter=',', names=True, case_sensitive=True)
data = np.array(data.tolist()) # was an array of arrays earlier
print(data.shape)
x = np.copy(data[:,0:8]) #features
y = np.copy(data[:,8]) #targets
features = 8 #or number of qubits which I kept equal to number of features in this case

(768, 9)


In [3]:
from sklearn.model_selection import train_test_split
import qiskit
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.multiclass_extensions import one_against_rest, all_pairs
from qiskit.aqua.components.feature_maps import SecondOrderExpansion
#from qiskit.aqua.input import ClassificationInput
#from qiskit.aqua import run_algorithm

In [5]:
#please and please again, use your own IBMQ token (can be generated easily), I have very few processes left for me.
key = '243ff4202efc1b45ad9282b9d6d524cdcccfc25561a86de37424b5ddd9aaa72969ea300500423019c48da10777e3241a5a45ff4a223439239cba4bed8d8709cc'

In [6]:
qiskit.IBMQ.save_account(key)
provider = qiskit.IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')

configrc.store_credentials:WARNING:2020-06-16 13:30:14,519: Credentials already present. Set overwrite=True to overwrite.


In [8]:
#using only first 100, if anyone is brave enough, train_test_split(x, y, test_size=0.2)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2)

training_data = {'A':train_x[train_y==0],'B':train_x[train_y==1]}
testing_data = {'A':test_x[test_y==0],'B':test_x[test_y==1]}

In [9]:
#Using RawFeatureVector reduces time immensely!
#feature_map = SecondOrderExpansion(feature_dimension=features,depth=2,entanglement='full')
from qiskit.aqua.components.feature_maps import RawFeatureVector
feature_map = RawFeatureVector(8)

svm = QSVM(feature_map,training_data,testing_data)
shots = 1 
quantum_instance = QuantumInstance(backend,shots=shots,skip_qobj_validation=False)

In [13]:
!pip install cvxopt

     |████████████████████████████████| 11.6 MB 6.2 MB/s eta 0:00:01


In [10]:
%%time
result = svm.run(quantum_instance)

CPU times: user 1h 35min 56s, sys: 8min 30s, total: 1h 44min 26s
Wall time: 6h 44min 34s


In [11]:
print(result)

{'kernel_matrix_training': array([[1., 0., 0., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       [0., 1., 1., ..., 0., 1., 0.],
       ...,
       [1., 1., 0., ..., 1., 0., 1.],
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.]]), 'svm': {'alphas': array([3.84157702e+02, 1.78411207e+02, 3.17052767e+02, 3.52630557e+01,
       3.95742243e+01, 4.29433439e+01, 1.47224338e+02, 2.51176596e+02,
       4.22106664e+02, 2.33135853e+02, 2.65684971e+02, 2.20864310e+02,
       2.26702372e+02, 2.18416176e+02, 3.56675089e+02, 2.52306392e+02,
       2.84524068e+02, 3.51787823e+01, 1.11917235e+02, 4.71798889e+02,
       3.71969298e+02, 6.59971291e+01, 5.78713377e+02, 4.27785871e+02,
       5.45377710e+02, 1.33947597e+02, 3.26590590e+02, 6.06964095e+02,
       4.65469459e+02, 2.43507845e+02, 3.84107935e+02, 1.38583153e+02,
       3.98063346e+02, 1.49162946e+02, 4.72159459e+02, 3.49092922e+01,
       2.06451366e+01, 3.28328317e+02, 7.69795944e+01, 5.60201443e+01,
   